In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import checklist
import spacy
import itertools

In [125]:
import checklist.editor
import checklist.text_generation
from checklist.mft import Mft
from checklist.inv_dir import Inv, Dir
from checklist.expect import Expect
import numpy as np

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
tg = checklist.text_generation.TextGenerator(nlp)

In [6]:
from checklist.pred_wrapper import PredictorWrapper

In [693]:
import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
sentiment = model_wrapper.ModelWrapper()
pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)

In [10]:
# tg.unmask_multiple(['I really <mask> the pilot.', 'I really <mask> the flight.'], metric='min')

In [13]:
import pickle
editor = checklist.editor.Editor()
c = pickle.load(open('/home/marcotcr/work/ml-tests/data/common.pkl', 'rb'))
editor.lexicons['male'] = c['male_names']
editor.lexicons['female'] = c['female_names']
c.keys()


In [15]:
nouns = ['flight', 'seat', 'pilot', 'staff', 'plane', 'airline', 'cabin crew', 'aircraft', 'food']
pos = ['liked', 'enjoyed', 'loved', 'admired', 'appreciated']
neg = ['hated', 'disliked', 'regretted']


In [16]:
ts = editor.template('I really <mask> the {n}.', n=nouns)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000)][:20])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['like', 'liked', 'love', 'enjoyed', 'loved', 'appreciate', 'enjoy', 'appreciated', 'miss', 'Like', 'missed', 'needed', 'hate', 'wanted', 'likes', 'got', 'admired', 'admire', 'hated', 'recommend']


In [17]:
ts = editor.template('<mask> really {pn} the {n}.', n=nouns, pn=pos+neg)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000)][:20])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['I', 'We', 'He', 'he', 'She', 'They', 'we', 'and', '...', 'she', 'they', 'Everyone', 'i', 'People', 'You', 'Alex', 'Everybody', 'And', 'Dad', 'John']


In [18]:
subj = ['I', 'We', 'They', 'we', 'He', 'he', 'She', 'she', 'they', 'people', 'People', 'you', '']

In [19]:
ts = editor.template('I didn\'t like the {n}, it was very <mask>.', n=nouns)
print([x[0][0] for x in tg.unmask_multiple(ts, beam_size=1000, metric='avg')][:100])
# tg.unmask_multiple(ts, metric='avg', beam_size=100)

['bad', 'boring', 'strange', 'cold', 'slow', 'unpleasant', 'weird', 'difficult', 'uncomfortable', 'expensive', 'rough', 'noisy', 'ugly', 'poor', 'dark', 'different', 'small', 'disappointing', 'quiet', 'rude', 'confusing', 'dull', 'sad', 'odd', 'loud', 'annoying', 'simple', 'harsh', 'short', 'unusual', 'scary', 'hard', 'nice', 'hot', 'dangerous', 'aggressive', 'heavy', 'dirty', 'conservative', 'old', 'tough', 'bland', 'low', 'cheap', 'frustrating', 'depressing', 'nasty', 'long', 'crude', 'horrible', 'stupid', 'complicated', 'funny', 'fast', 'good', 'basic', 'weak', 'wrong', 'awkward', 'terrible', 'negative', 'crowded', 'plain', 'thin', 'interesting', 'awful', 'bizarre', 'intimidating', 'cool', 'primitive', 'unstable', 'flat', 'narrow', 'stressful', 'stiff', 'empty', 'chaotic', 'light', 'irritating', 'big', 'unfair', 'sick', 'similar', 'angry', 'mean', 'predictable', 'easy', 'offensive', 'foreign', 'violent', 'tight', 'comfortable', 'busy', 'challenging', 'arrogant', 'disgusting', 'ineff

In [20]:
pos_adj = ['good', 'nice', 'helpful', 'comfortable', 'cool', 'efficient', 'pleasant', 'interesting', 'impressive', 'welcoming', 'professional', 'beautiful', 'exciting', 'positive', 'solid', 'amazing', 'wonderful', 'lovely']
neg_adj = ['bad', 'boring', 'unpleasant', 'difficult', 'uncomfortable', 'ugly', 'poor', 'disappointing', 'sad', 'annoying', 'dirty', 'frustrating', 'depressing', 'nasty', 'horrible', 'stupid', 'negative', 'awful', 'stressful', 'irritating', 'disgusting']

In [21]:
pos_eg = editor.template('{sub}{really} {p} the {n}.', sub=subj, really=[' really', ''], n=nouns, p=pos)
pos_eg += editor.template('The {n} was{very} {p}.', sub=subj, very=[' very', ''], n=nouns, p=pos_adj)
neg_eg = editor.template('{sub}{really} {p} the {n}.', sub=subj, really=[' really', ''], n=nouns, p=neg)
neg_eg += editor.template('The {n} was{very} {p}.', sub=subj, very=[' very', ''], n=nouns, p=neg_adj)

In [22]:
len(pos_eg), len(neg_eg)

(1494, 1080)

In [368]:
test_pos = Mft(pos_eg, labels=1)
test_neg = Mft(neg_eg, labels=0)

In [371]:
# test_pos.set_expect(Expect.single(Expect.eq()))

In [372]:
test_pos.run(pp)
test_neg.run(pp)

Predicting 1494 examples
Predicting 1080 examples


In [410]:
test_neg.summary(n=5)

Test cases:     1080
Failure rate:   1.7%

Example fails:
1 (1.0) The cabin crew was very stressful.
1 (1.0) The flight was very stressful.
1 (1.0) The flight was stressful.
1 (1.0) The aircraft was stressful.
1 (1.0) The seat was very stressful.


In [412]:
test_pos.summary(n=5)

Test cases:     1494
Failure rate:   0.0%


## Fairness

### As invariance

In [701]:
adj = [x[0][0] for x in tg.unmask('John is a black <mask>.')]

In [720]:
races = ['black', 'white', 'hispanic', 'asian']
d = editor.template(['{male} is a %s {adj}.' % r for r in races], adj=adj, return_maps=False)

In [713]:
test = Inv(d, threshold=0.1)
test.run(pp, overwrite=True)

Predicting 8000 examples


In [714]:
test.summary(n=3)

Test cases:     2000
Failure rate:   95.0%

Example fails:
0 (0.1) Gabriel is a black Muslim.
1 (0.9) Gabriel is a hispanic Muslim.
1 (0.9) Gabriel is a asian Muslim.

0 (0.0) James is a black person.
1 (0.9) James is a hispanic person.
1 (0.9) James is a asian person.

0 (0.1) Jason is a black girl.
1 (1.0) Jason is a hispanic girl.
1 (1.0) Jason is a asian girl.



In [727]:
pps = np.array([[x[i][1] for i in range(4)] for x in test.results.confs])
for i in range(4):
    print('%.2f %s' % (pps[:, i].mean(), races[i]))

0.12 black
0.19 white
0.96 hispanic
0.98 asian


### as MFT

In [741]:
d, m = editor.template('{male} is a {r} {adj}.', r=races,  adj=adj, return_maps=True, nsamples=100)

In [731]:
m[0]

{'r': 'black', 'adj': 'man', 'male': 'Michael'}

In [773]:
np.array([None])

array([None], dtype=object)

In [777]:
import collections
def is_fair(data, preds, confs, labels, metas):
    rets = collections.defaultdict(lambda: [])
    ret = np.repeat(None, len(data))
    for c, m in zip(confs, metas):
        rets[m['r']].append(c[1])
    for i, r in enumerate(rets):
        rets[r] = np.mean(rets[r])
        ret[i] = rets[r]
    print(rets)
    return ret 


In [778]:
ex = Expect.test(is_fair)
test = Mft(d, expect=ex, meta=m)
test.run(pp, overwrite=True)

Predicting 100 examples
defaultdict(<function is_fair.<locals>.<lambda> at 0x7f126044f598>, {'white': 0.356781789360361, 'asian': 0.9733978605270386, 'hispanic': 0.9593281026544243, 'black': 0.12038368929643184})


This is a bad idea, fairness tests should just be INVs

## Inv, dir

In [413]:
import csv
r = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))
labels = []
confs = []
airlines = []
tdata = []
reasons = []
for row in r:
    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']
    labels.append(sentiment)
    confs.append(conf)
    airlines.append(airline)
    tdata.append(text)
    reasons.append(row['negativereason'])

mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
labels = np.array([mapping[x] for x in labels]).astype(int)

In [414]:
sentences = tdata
parsed_data = list(nlp.pipe(sentences))

In [415]:
def add_typo(string):
    string = list(string)
    swaps = 1
    swaps = np.random.choice(len(string) - 1, swaps)
    for swap in swaps:
        swap = np.random.choice(len(string) - 1)
        tmp = string[swap]
        string[swap] = string[swap + 1]
        string[swap + 1] = tmp
    return ''.join(string)

def add_typos(string):
    return list(set([add_typo(string) for _ in range(10)]))


In [416]:
from checklist.perturb import Perturb

In [417]:
data = Perturb.perturb(np.random.choice(sentences, 100), add_typos, keep_original=True)

In [418]:
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)

Predicting 1043 examples


In [433]:
test.summary(n=3)

Test cases:     100
Failure rate:   24.0%

Example fails:
0.0 @JetBlue Not for the dates or destination I'm headed 😔
0.5 @JetBlu eNot for the dates or destination I'm headed 😔

0.5 @AmericanAir will do. I also passed the website around to other passengers.
0.7 @AmericanAir will do. I also passed the website around to other psasengers.
0.7 @AmericanAir will do. I also passed the ewbsite around to other passengers.

1.0 @united DM'ed you
0.1 @uinted DM'ed you
0.1 @united DM'ed yuo



In [435]:
data = Perturb.perturb(np.random.choice(sentences, 1000), add_typo, keep_original=True)
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)

Predicting 2000 examples


In [436]:
test.summary(n=3)

Test cases:     1000
Failure rate:   4.9%

Example fails:
0.7 @united when will you offer real food in american clubs like the amazing food you offer in Heathrow?
0.2 @uinted when will you offer real food in american clubs like the amazing food you offer in Heathrow?

1.0 @united maybe on my return trip 👍
0.1 @united maybe on my erturn trip 👍

0.7 @united was in the air. Just DMd you
0.0 @united was in the ai.r Just DMd you



In [442]:
# TODO: get from somethwere else
def add_negatives(string):
    string = string.strip('.')
    return [string + '. ' + l for l in ['I hate you', 'I despise you', 'You suck']]
def add_positive(string):
    string = string.strip('.')
    return [string + '. ' + l for l in ['I love you', 'I like you', 'You are great']]


In [791]:
data = Perturb.perturb(np.random.choice(sentences, 1000), add_negatives, keep_original=True)
expect_fn = Expect.monotonic(1, increasing=False, tolerance=0.1)
test = Dir(data, expect_fn)
test.run(pp, overwrite=True)
test.set_monotonic_print(label=1, increasing=False)
test.summary(3)

Predicting 4000 examples
Test cases:     1000
Filtered cases: 259 (25.9%)
Failure rate:   0.0%


In [806]:
def wrapper2(fn, check):
    def fzz():
        msg = fn()
        print('fzz', msg, check)
    return fzz
def wrapper(fn, check=True):
    def fz():
        msg = fn() + 'fz'
        return msg
    return wrapper2(fz, check)
    
def f1(label=None, un=3, a=3, b=3):
    def f():
        nonlocal label
        if label is None:
            label = 3
        return str(label) + str(un)
    return wrapper(f)

In [807]:
f1(3)()

fzz 33fz True


In [818]:
tg.unmask('Mary is not a man, she is a <mask>.')

[(['woman'], ' Mary is not a man, she is a woman.', 17.750459671020508),
 (['child'], ' Mary is not a man, she is a child.', 14.719247817993164),
 (['monster'], ' Mary is not a man, she is a monster.', 14.308874130249023),
 (['girl'], ' Mary is not a man, she is a girl.', 13.8037109375),
 (['beast'], ' Mary is not a man, she is a beast.', 13.60313606262207),
 (['machine'], ' Mary is not a man, she is a machine.', 13.506260871887207),
 (['god'], ' Mary is not a man, she is a god.', 13.43105411529541),
 (['God'], ' Mary is not a man, she is a God.', 13.350319862365723),
 (['man'], ' Mary is not a man, she is a man.', 13.242627143859863),
 (['ghost'], ' Mary is not a man, she is a ghost.', 13.10810661315918),
 (['demon'], ' Mary is not a man, she is a demon.', 13.078417778015137),
 (['creature'], ' Mary is not a man, she is a creature.', 13.015871047973633),
 (['slave'], ' Mary is not a man, she is a slave.', 12.9219970703125),
 (['goddess'], ' Mary is not a man, she is a goddess.', 12.87

In [823]:
tg.unmask('Mary is a <mask>.')

[(['joke'], ' Mary is a joke.', 10.655401229858398),
 (['woman'], ' Mary is a woman.', 10.290670394897461),
 (['problem'], ' Mary is a problem.', 10.219472885131836),
 (['man'], ' Mary is a man.', 10.078128814697266),
 (['metaphor'], ' Mary is a metaphor.', 9.863449096679688),
 (['lie'], ' Mary is a lie.', 9.859661102294922),
 (['myth'], ' Mary is a myth.', 9.854726791381836),
 (['story'], ' Mary is a story.', 9.853984832763672),
 (['girl'], ' Mary is a girl.', 9.726561546325684),
 (['mess'], ' Mary is a mess.', 9.668272018432617),
 (['mistake'], ' Mary is a mistake.', 9.623100280761719),
 (['shame'], ' Mary is a shame.', 9.5949068069458),
 (['boy'], ' Mary is a boy.', 9.545333862304688),
 (['dream'], ' Mary is a dream.', 9.408576965332031),
 (['monster'], ' Mary is a monster.', 9.394660949707031),
 (['fact'], ' Mary is a fact.', 9.374675750732422),
 (['pseudonym'], ' Mary is a pseudonym.', 9.32861328125),
 (['robot'], ' Mary is a robot.', 9.310115814208984),
 (['tragedy'], ' Mary is a

In [834]:
tg.unmask('John <mask> a <mask> man.')

[(['was', 'young'], ' John was a young man.', 14.495121955871582),
 (['died', 'free'], ' John died a free man.', 14.45598316192627),
 (['was', 'good'], ' John was a good man.', 14.438433170318604),
 (['is', 'good'], ' John is a good man.', 14.233307838439941),
 (['died', 'young'], ' John died a young man.', 14.056387901306152),
 (["'s", 'good'], " John's a good man.", 14.03753137588501),
 (['is', 'young'], ' John is a young man.', 13.927804470062256),
 (['was', 'black'], ' John was a black man.', 13.852426528930664),
 (['was', 'great'], ' John was a great man.', 13.799434185028076),
 (['was', 'free'], ' John was a free man.', 13.700941562652588),
 (['was', 'happy'], ' John was a happy man.', 13.667884826660156),
 (['was', 'rich'], ' John was a rich man.', 13.655069828033447),
 (['was', 'beautiful'], ' John was a beautiful man.', 13.651923179626465),
 (['is', 'great'], ' John is a great man.', 13.616724967956543),
 (['was', 'troubled'], ' John was a troubled man.', 13.616347789764404),


In [828]:
tg.unmask_multiple(['John is a <mask>.', 'John is not a <mask>.', 'Mary is a <mask>.', 'Mary is not a man, she is a <mask>.'], metric='avg', beam_size=1000)[:10]

[(['woman'], ' Mary is not a man, she is awoman.', 13.98249340057373),
 (['man'], ' Mary is not a man, she is a man.', 10.98160719871521),
 (['monster'], ' Mary is not a man, she is a monster.', 10.796409130096436),
 (['girl'], ' Mary is not a man, she is a girl.', 10.478504419326782),
 (['joke'], ' Mary is not a man, she is a joke.', 10.465309858322144),
 (['myth'], ' Mary is not a man, she is a myth.', 10.431484699249268),
 (['god'], ' Mary is not a man, she is a god.', 10.417447566986084),
 (['child'], ' Mary is not a man, she is a child.', 10.40675973892212),
 (['robot'], ' Mary is not a man, she is a robot.', 10.308293581008911),
 (['lie'], ' Mary is not a man, she is a lie.', 10.136309623718262)]

In [808]:
data = Perturb.perturb(np.random.choice(sentences, 100), add_positive, keep_original=True)
expect_fn = Expect.monotonic(1, increasing=True, tolerance=0.1)
test = Dir(data, expect_fn)
test.run(pp, overwrite=True)
test.set_monotonic_print(label=1, increasing=True)
test.summary(3)

Predicting 400 examples
Test cases:     100
Filtered cases: 72 (72.0%)
Failure rate:   0.0%


### QQP

In [509]:
qqp = model_wrapper.ModelWrapper()
pp = PredictorWrapper.wrap_softmax(qqp.predict_proba)

In [166]:
qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

"	What does the Quran say about homosexuality?	0



In [167]:
quran = [x[0][0] for x in tg.unmask('What does the Quran say about <mask>?')]

In [168]:
quran.remove('homosexuals')
quran.remove('gays')
quran.remove('this')

In [515]:
data = editor.template(('What does the Quran say about {thing1}?', 'What does the Quran say about {thing2}?'),
                thing1=quran, thing2=quran, remove_duplicates=True)
test = Mft(data, labels=0)
test.run(pp, overwrite=True)
test.summary(n=5)

Predicting 272 examples
Test cases:     272
Failure rate:   2.2%

Example fails:
0.6 ('What does the Quran say about polygamy?', 'What does the Quran say about circumcision?')
0.7 ('What does the Quran say about circumcision?', 'What does the Quran say about polygamy?')
0.8 ('What does the Quran say about Muhammad?', 'What does the Quran say about Muslims?')
1.0 ('What does the Quran say about Islam?', 'What does the Quran say about Muslims?')
0.8 ('What does the Quran say about Muslims?', 'What does the Quran say about Islam?')


In [171]:
print([x[0][0] for x in tg.unmask_multiple(editor.template('What does the <mask> say about {thing}?', thing=quran))])

['Bible', 'book', 'report', 'survey', 'poll', 'study', 'Constitution', 'data', 'bible', 'election', 'census', 'law', 'UN', 'Quran', 'evidence', 'science', 'constitution', 'Koran', 'vote', 'bill', 'film', 'Holocaust', 'research', 'Pope', 'letter', 'article', 'verdict', 'world', 'movie', 'government', 'video', 'Torah', 'song', 'moon', 'public', 'dictionary', 'president', 'referendum', 'ACA', 'eclipse', 'church', 'US', 'media', 'Church', 'pope', 'record', 'test', 'answer', 'literature', 'Prophet', 'CDC', 'FDA', 'WHO', 'scripture', 'community', 'Buddha', 'ban', 'prophet', 'statement', 'interview', 'Gospel', 'Book', 'past', 'question', 'decision']


In [516]:
books = ['Bible', 'Constitution', 'Quran', 'Pope', 'Torah', 'Church', 'Buddha', 'Prophet', 'Gospel', 'Book']
data = editor.template(('What does the {book1} say about {thing}?', 'What does the {book2} say about {thing}?'),
                thing=quran, book1=books, book2=books, remove_duplicates=True)
test = Mft(data, labels=0)
test.run(pp, overwrite=True)
test.summary(n=5)

Predicting 1530 examples
Test cases:     1530
Failure rate:   8.9%

Example fails:
0.6 ('What does the Bible say about homosexuality?', 'What does the Church say about homosexuality?')
1.0 ('What does the Gospel say about you?', 'What does the Bible say about you?')
0.9 ('What does the Quran say about religion?', 'What does the Prophet say about religion?')
0.6 ('What does the Bible say about women?', 'What does the Torah say about women?')
1.0 ('What does the Bible say about God?', 'What does the Gospel say about God?')


### Inv, dirs

In [174]:
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))

In [175]:
processed_qs = [spacy_map[x[0]] for x in qs] + [spacy_map[x[1]] for x in qs]

In [176]:
processed_pairs = [(spacy_map[x[0]],spacy_map[x[1]]) for x in qs]

In [526]:
def change_names(pair):
    x0, x1 = pair
    ents_0 = set([x[0].text for x in x0.ents if x[0].ent_type_ == 'PERSON'])
    ents_1 = set([x[0].text for x in x1.ents if x[0].ent_type_ == 'PERSON'])
    ret = []
    if ents_0 and ents_0.intersection(ents_1):
        for e in ents_0.intersection(ents_1):
            if e == 'Quora':
                continue
            ret.extend([(pair[0].text.replace(e, n), pair[1].text.replace(e, n)) for n in editor.lexicons['male'] + editor.lexicons['female']])
    if ret:
        idxs = np.random.choice(len(ret), min(5, len(ret)), replace=False)
        return [ret[i] for i in idxs]
# change_names(s)

In [530]:
idxs = np.random.choice(len(processed_pairs), 2000,  replace=False)
# idxs = np.random.choice(len(processed_pairs), len(processed_pairs),  replace=False)
sl = [processed_pairs[i] for i in idxs]
data = Perturb.perturb(sl, change_names, keep_original=True)
# test = Dir(data, Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=True, tolerance=0)))) )
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)
test.summary(n=3)

Predicting 9792 examples
Test cases:     1632
Failure rate:   11.1%

Example fails:
0.2 ("What is India's relationship with Bangladesh?", "What is Bangladesh's relationship with India?")
0.9 ("What is India's relationship with Isabella?", "What is Isabella's relationship with India?")
0.9 ("What is India's relationship with Jesus?", "What is Jesus's relationship with India?")

1.0 ('Which is best place to stay and visit in Kerala?', 'What are the best 10 places to visit in Kerala including any falls?')
0.4 ('Which is best place to stay and visit in Sara?', 'What are the best 10 places to visit in Sara including any falls?')
0.4 ('Which is best place to stay and visit in Emily?', 'What are the best 10 places to visit in Emily including any falls?')

1.0 ('What is Jake Williams’s history that made him into a narcissist?', 'How is Jake Williams a narcissist?')
0.3 ('What is Sara Williams’s history that made him into a narcissist?', 'How is Sara Williams a narcissist?')
0.4 ('What is Nicol

In [542]:

idxs = np.random.choice(len(processed_pairs), 2000,  replace=False)
# idxs = np.random.choice(len(processed_pairs), len(processed_pairs),  replace=False)
sl = [processed_pairs[i] for i in idxs]
data = Perturb.perturb(sl, change_names, keep_original=True)
# test = Dir(data, Expect.wrap(Expect.all(Expect.pairwise_to_group(Expect.monotonic_label(1, increasing=True, tolerance=0)))) )
test = Inv(data, threshold=0.1)
test.run(pp, overwrite=True)
test.summary(n=3)

Predicting 432 examples
Test cases:     72
Failure rate:   5.6%

Example fails:
1.0 ('What are the best places to visit in Wayanad, Kerala?', 'What can be the medium budget to visit best places in Kerala for three members (2-3 days)?')
0.2 ('What are the best places to visit in Wayanad, Isaiah?', 'What can be the medium budget to visit best places in Isaiah for three members (2-3 days)?')
0.0 ('What are the best places to visit in Wayanad, Jamie?', 'What can be the medium budget to visit best places in Jamie for three members (2-3 days)?')

0.9 ('Was Donald Trump always rich?', 'How rich is Donald Trump?')
0.5 ('Was Jason Trump always rich?', 'How rich is Jason Trump?')

1.0 ('What is the best way to start contributing to the Linux kernel?', 'How should I start contributing for Linux?')
0.2 ('What is the best way to start contributing to the Gabriel kernel?', 'How should I start contributing for Gabriel?')
0.0 ('What is the best way to start contributing to the Elizabeth kernel?', 'How

In [186]:
# test.results.passed

In [531]:
def change_name_in_one(pair):
    x0, x1 = pair
    ents_0 = set([x[0].text for x in x0.ents if x[0].ent_type_ == 'PERSON'])
    ents_1 = set([x[0].text for x in x1.ents if x[0].ent_type_ == 'PERSON'])
    ret = []
    if ents_0 and ents_0.intersection(ents_1):
        for e in ents_0.intersection(ents_1):
            if e == 'Quora':
                continue
            ret.extend([(pair[0].text.replace(e, n), pair[1].text) for n in editor.lexicons['male'] + editor.lexicons['female']])
            ret.extend([(pair[0].text, pair[1].text.replace(e, n)) for n in editor.lexicons['male'] + editor.lexicons['female']])
    if ret:
        idxs = np.random.choice(len(ret), min(5, len(ret)), replace=False)
        return [ret[i] for i in idxs]
# change_names(s)

In [557]:
monotonic = Expect.monotonic(1, increasing=False, tolerance=0.1)
idxs = np.random.choice(len(processed_pairs), 3000, replace=False)
sl = [processed_pairs[i] for i in idxs]
data = Perturb.perturb(sl, change_name_in_one, keep_original=True)
test = Dir(data, monotonic)
test.run(pp, overwrite=True)
test.set_monotonic_print(label=1, increasing=False)
test.summary(n=3)
# test.results.passed[test.results.passed != None].astype(bool).mean()

Predicting 756 examples
Test cases:     126
Filtered cases: 64 (50.8%)
Failure rate:   3.2%

Example fails:
0.1 ('Why does Donald Trump think the debate schedule favors Hillary?', 'Why is Donald Trump saying the debate schedule is unfair? Is it because his support base would rather watch the NFL than his debate?')
0.2 ('Why does Donald Trump think the debate schedule favors Hillary?', 'Why is Ava Trump saying the debate schedule is unfair? Is it because his support base would rather watch the NFL than his debate?')

0.5 ('What do you feel about Donald Trump winning the elections?', 'How do you feel about Donald Trump winning the Republican nomination?')
1.0 ('What do you feel about Donald Trump winning the elections?', 'How do you feel about Adam Trump winning the Republican nomination?')
1.0 ('What do you feel about Donald Trump winning the elections?', 'How do you feel about Austin Trump winning the Republican nomination?')



In [558]:
# if only consider examples that were duplicates
sl_fn = lambda pred, *args, **kwargs: pred == 1
# This substitutes the previous one
# def sl_fn(x, pred, *args, **kwargs):
#     print(pred)
#     return pred[0] == 1
sl_fn2 = lambda x, pred, *args, **kwargs: pred[0] == 1
is_false = Expect.single(Expect.eq(0), agg_fn=Expect.all(ignore_first=True))#, slice_fn=sl_fn)
# is_false = Expect.slice_pairwise(is_false, sl_fn)
is_false = Expect.slice_testcase(is_false, sl_fn2)
test.set_expect(is_false)

In [575]:
from checklist.inv_dir import pairwise_print_fn
def fail_cr(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    return pred != 0
def sort_cr(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    return orig_conf[0]
print_fn = pairwise_print_fn(fail_cr, sort_cr)
test.summary(n=3, print_fn=print_fn)

Test cases:     126
Filtered cases: 53 (42.1%)
Failure rate:   52.1%

Example fails:
1.0 ('What are the best books on Joseph Goebbels?', 'Are there any really, really interesting books on Joseph Goebbels?')
1.0 ('What are the best books on Peter Goebbels?', 'Are there any really, really interesting books on Joseph Goebbels?')
1.0 ('What are the best books on Jesse Goebbels?', 'Are there any really, really interesting books on Joseph Goebbels?')

1.0 ('How cold can the Gobi Desert get, and how do its average temperatures compare to the ones in the Sahara?', 'How cold can the Gobi Desert get, and how do its average temperatures compare to the ones in the Sonoran Desert?')
1.0 ('How cold can the Gobi Desert get, and how do its average temperatures compare to the ones in the Sahara?', 'How cold can Tiffany Gobi Desert get, and how do its average temperatures compare to Tiffany ones in Tiffany Sonoran Desert?')
1.0 ('How cold can Ella Gobi Desert get, and how do its average temperatures com

## Squad

In [576]:
from mltests import bert_squad_model

In [577]:
model = bert_squad_model.BertSquad()

In [579]:
# Just makes confidence=1 for every prediction
pp = PredictorWrapper.wrap_predict(model.predict_pairs)

In [580]:
model.predict_pairs([('Who is dumb?', 'Mary is somewhat dumb. John is not so dumb.')])

['Mary']

In [581]:
pp([('Who is dumb?', 'Mary is somewhat dumb. John is not so dumb.')])

(['Mary'], array([1.]))

In [582]:
# editor.lexicons

In [583]:
verbs = [x[0][0] for x in tg.unmask('Luke told Mary that she should probably stop <mask>.')]
verbs = [x for x in verbs if 'ing' in x]
print(verbs)

['drinking', 'smoking', 'crying', 'reading', 'writing', 'worrying', 'eating', 'driving', 'listening', 'praying', 'talking', 'working', 'trying', 'calling', 'laughing']


In [584]:
labels = []
data, maps = editor.template(('Who is currently {verb}?', '{male} told {female} she should probably stop {verb}'), nsamples=100, verb=verbs, return_maps=True)
labels += [m['female'] for m in maps]
n, maps = editor.template(('Who is currently {verb}?', '{male} told {female} he should probably stop {verb}'), nsamples=100,verb=verbs, return_maps=True)
labels += [m['male'] for m in maps]
data += n
n, maps = editor.template(('Who is currently {verb}?', '{female} told {male} he should probably stop {verb}'), nsamples=100, verb=verbs, return_maps=True)
labels += [m['male'] for m in maps]
data += n
n, maps = editor.template(('Who is currently {verb}?', '{female} told {male} she should probably stop {verb}'), nsamples=100, verb=verbs, return_maps=True)
labels += [m['female'] for m in maps]
data += n

In [585]:
data[0]

('Who is currently worrying?',
 'Juan told Crystal she should probably stop worrying')

In [586]:
test = Mft(data, labels=labels)
test.run(pp)
# test.results.passed.mean()

Predicting 400 examples


In [609]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    q, c = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [613]:
def format_squad(x, pred, conf, label=None, *args, **kwargs):
    q, c = x
    ret = 'Q: %s\n' % (q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [615]:
test.summary(n=3, format_example_fn=format_squad_with_context)

Test cases:     400
Failure rate:   58.2%

Example fails:
C: Taylor told Carlos she should probably stop praying
Q: Who is currently praying?
A: Taylor
P: Carlos

C: Carlos told Rachel he should probably stop worrying
Q: Who is currently worrying?
A: Carlos
P: Rachel

C: Peter told Chloe she should probably stop trying
Q: Who is currently trying?
A: Chloe
P: Peter



In [616]:
liked = [x[0][0] for x in tg.unmask('John was told by Luke that he really likes <mask>.', beam_size=100)]

In [617]:
labels = []
data, maps = editor.template(('Who really likes {liked}?', '{male1} was told by {male2} that {male2} really likes {liked}.'), male1=editor.lexicons['male'], male2=editor.lexicons['male'], nsamples=100, verb=verbs, liked=liked, return_maps=True, remove_duplicates=True)
labels += [m['male2'] for m in maps]

In [618]:
test = Mft(data, labels=labels)
test.run(pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 100 examples



Test cases:     100
Failure rate:   44.0%

Example fails:
C: Eric was told by Austin that Austin really likes Hunter.
Q: Who really likes Hunter?
A: Austin
P: Eric

C: Ethan was told by Juan that Juan really likes Jess.
Q: Who really likes Jess?
A: Juan
P: Juan that Juan

C: Robert was told by Ian that Ian really likes Vanessa.
Q: Who really likes Vanessa?
A: Ian
P: Robert



In [713]:
print([x[0][0] for x in tg.unmask_multiple(editor.template('What does the <mask> say about {thing}?', thing=quran))])

['Bible', 'book', 'report', 'survey', 'poll', 'study', 'Constitution', 'data', 'bible', 'election', 'census', 'law', 'UN', 'Quran', 'evidence', 'science', 'constitution', 'Koran', 'vote', 'bill', 'film', 'Holocaust', 'research', 'Pope', 'letter', 'article', 'verdict', 'world', 'movie', 'government', 'video', 'Torah', 'song', 'moon', 'public', 'dictionary', 'president', 'referendum', 'ACA', 'eclipse', 'church', 'US', 'media', 'Church', 'pope', 'record', 'test', 'answer', 'literature', 'Prophet', 'CDC', 'FDA', 'WHO', 'scripture', 'community', 'Buddha', 'ban', 'prophet', 'statement', 'interview', 'Gospel', 'Book', 'past', 'question', 'decision']


In [619]:
import json
def load_squad(fold='validation'):
    answers = []
    data = []
    ids = []
    files = {
        'validation': '/home/marcotcr/datasets/squad/dev-v1.1.json',
        'train': '/home/marcotcr//datasets/squad/train-v1.1.json',
        }
    f = json.load(open(files[fold]))
    for t in f['data']:
        for p in t['paragraphs']:
            context = p['context']
            for qa in p['qas']:
                data.append({'passage': context, 'question': qa['question'], 'id': qa['id']})
                answers.append(set([(x['text'], x['answer_start']) for x in qa['answers']]))
    return data, answers


In [620]:
data, answers =  load_squad()

In [621]:
pairs = [(x['question'], x['passage']) for x in data]

In [622]:
import re

In [626]:
def change_name(x):
    q, c = x
    in_p = set()
    not_in_p = set()
    for n in editor.lexicons['male'][:10]:
        if re.search(r'\b%s\b' % n, c):
            in_p.add(n)
        else:
            not_in_p.add(n)
    if not in_p:
        return None
    ret = []
    ret_add = []
    for p in in_p:
        for n in not_in_p:
            ret.append((re.sub(r'\b%s\b' % p, n, q), re.sub(r'\b%s\b' % p, n, c)))
            ret_add.append((p, n))
    if ret:
        idxs = np.random.choice(len(ret), min(5, len(ret)), replace=False)
        ret = [ret[i] for i in idxs]
        ret_add = [ret_add[i] for i in idxs]
    return ret, ret_add

In [273]:
# [i for i, x in enumerate(pairs) if 'John' in x[1]]

In [692]:
def new_eq(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    if meta:
        p, n = meta
        orig_pred =  re.sub(r'\b%s\b' % p, n, orig_pred)
    return pred == orig_pred

def format_name(x, pred, conf, label=None, meta=None):
    ret = ''
    if meta is not None and len(meta):
        ret = '%s -> %s\n' % meta
    ret += format_squad(x, pred, conf, label, meta)
    return ret
    
# tt = Expect.wrap(Expect.all(Expect.pairwise_to_group(new_eq), ignore_first=True))
random_idxs = np.random.choice(len(pairs), 3000)
data, meta = Perturb.perturb([pairs[i] for i in random_idxs], change_name, returns_additional=True)
tt = Expect.pairwise(new_eq)
test = Inv(data, expect=tt, meta=meta)
test.run(pp)
test.summary(n=3, format_example_fn=format_name)

Predicting 2058 examples



Test cases:     343
Failure rate:   1.2%

Example fails:
Q: What did John Dobson describe Newcastle as?
P: neoclassical centre referred to as Tyneside Classical

John -> Daniel
Q: What did Daniel Dobson describe Newcastle as?
P: neoclassical


Q: What Doctor was first referred to as "his secret"?
P: Eleventh Doctor

John -> James
Q: What Doctor was first referred to as "his secret"?
P: Eleventh Doctor meets an unknown incarnation of himself

John -> Joseph
Q: What Doctor was first referred to as "his secret"?
P: Eleventh Doctor meets an unknown incarnation of himself


Q: Ludwig Krapf recorded the name was what?
P: both Kenia and Kegnia

Joseph -> James
Q: Ludwig Krapf recorded the name was what?
P: Kenia and Kegnia

Joseph -> William
Q: Ludwig Krapf recorded the name was what?
P: Kenia and Kegnia




In [877]:
test.meta[0]

[{},
 ('John', 'Joshua'),
 ('John', 'Matthew'),
 ('John', 'William'),
 ('John', 'Christopher'),
 ('John', 'Michael'),
 ('John', 'James'),
 ('John', 'David'),
 ('John', 'Daniel'),
 ('John', 'Joseph')]

In [876]:
test.results.preds[0]

array(['John Fox', 'Joshua Fox', 'Matthew Fox', 'William Fox',
       'Christopher Fox', 'Michael Fox', 'James Fox', 'David Fox',
       'Daniel Fox', 'Joseph Fox'], dtype='<U15')

In [864]:
# meta[0]

In [861]:
# data[0]

<function checklist.expect.Expect.all.<locals>.expect(xs, preds, confs, labels=None, meta=None)>

In [853]:
change_name(pairs[178])[1][0]

('John', 'Joshua')

In [802]:
[x for x in pairs if 'John' in x[1]]

671

In [521]:
processed_qs[20]

How is vanilla extract made?

In [500]:
editor.lexicons['male']

['John', 'Luke', 'Mark']

In [498]:
len(processed_pairs)

40430

In [496]:
processed_qs[0].ents[0][0].ent_type_

'NORP'

In [440]:
# editor.template('This is {bad}',  bad=['bad', 'great', 'awesome'], return_maps=True)

In [14]:

editor.template(('This is {bad}', 'This is not {bad}'),  bad=['bad', 'great', 'awesome'], return_maps=False)

([('This is bad', 'This is not bad'),
  ('This is great', 'This is not great'),
  ('This is awesome', 'This is not awesome')],
 [{'bad': 'bad'}, {'bad': 'great'}, {'bad': 'awesome'}])

In [15]:
editor.template({
    'bad': 'This is {bad}',
    'notbad': 'This is not {bad}'},  bad=['bad', 'great', 'awesome'], return_maps=False)

[{'bad': 'This is bad', 'notbad': 'This is not bad'},
 {'bad': 'This is great', 'notbad': 'This is not great'},
 {'bad': 'This is awesome', 'notbad': 'This is not awesome'}]

In [57]:
a = {'bad': 'This is {bad}',
    'notbad': ('this is not {bad}', 'this is quite {abad}')}
b = ({'bad': 'This is {bad}', 'notbad': 'This is not {bad}'}, 'This is quite {bad}')
c = 'This is quite {bad}.'

In [64]:
editor.lexicons

{'male': ['John', 'Luke', 'Mark'], 'female': ['Mary', 'Judy', 'Julia']}

In [93]:
editor.template('This is {bad}, {male}', bad=('terrible', 'bad'), nsamples=None)

['This is terrible, John',
 'This is bad, John',
 'This is terrible, Luke',
 'This is bad, Luke',
 'This is terrible, Mark',
 'This is bad, Mark']